<a href="https://colab.research.google.com/github/arehfeldt/machine-learning-dump/blob/main/Aaron_Rehfeldt_CSE5522_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSE 5522 Lab #2: Sentiment Analysis**

The goals of this lab are to familarize you with:

*   Naive Bayes
*   Binary Classification
*   Data exploration
*   Working with text-based data (Tweets)

**Initial notes**

* (If you are using Google Colab) Make a copy of this page in your google drive so that you can edit it.

* While not completely necessary for this assignment, you may want to familiarize yourself with the following packages: [numpy](https://numpy.org), [scikit-learn](https://scikit-learn.org), [pandas](https://pandas.pydata.org), [matplotlib](https://matplotlib.org).
 * Especially numpy, many of the calculations in this (and later) lab can be done in one line using numpy. Whereas raw python may require 5-10x that.

* Feel free to (please do!) change the structure of the document below. Especially, add code sections to break your code into logical pieces and add text sections to explain your code or results

---
---

**Part 1: Hands-On #2 (0 pts)**

You will need to finish any remaining parts of the Hands-On that you didn't finish in class. Feel free to use code from the posted Hands-On #2 (partial) solution.

Note, you do not need to explicitly include the your Hands-On #2 solution here. But parts 2 and 3 assume you have a working solution, and you will probably need/want to copy code from it when you work on them.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

TweetUrl='https://github.com/aasiaeet/cse5522data/raw/master/db3_final_clean.csv'
tweet_dataframe=pd.read_csv(TweetUrl)

# wordDict maps words to id
# X is the document-word matrix holding the presence/absence of words in each tweet
wordDict = {}
idCounter = 0
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for word in allWords:
    if word not in wordDict:
      wordDict[word] = idCounter
      idCounter += 1

# initialize x and y
X = np.zeros((tweet_dataframe.shape[0], idCounter),dtype='float')
y = np.array(tweet_dataframe.iloc[:,2])

# fill in x by placing 1's corresponding to the presence of a word for each tweet
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for word in allWords:
    X[i, wordDict[word]]  = 1

print (X.shape, np.max(np.sum(X, axis=1)))

# compute_distrobutions:
#   x - test data features
#   y - true label of test data
#   returns the probability of a word being present in a tweet given that the tweet was positive/negative about the weather
def compute_distrobutions(x, y):
  # computes the probability of a word being present in a tweet given that the tweet was positive/negative about the weather
  probWordGivenPositive = sum(x[y > 0]) / sum(y > 0)
  probWordGivenNegative = sum(x[y < 0]) / sum(y < 0)

  # computes the prior probability of a tweet being positive or negative
  priorPositive = sum(y > 0) / y.size
  priorNegative = sum(y < 0) / y.size

  return probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative

def compute_log_probabilities(probability, minimum_probability):\
  # take care of 0 counts in our probability
  probability=np.where(probability>=minimum_probability,probability,minimum_probability)
  probability=np.where(probability<=(1-minimum_probability),probability,1-minimum_probability)

  return np.log(probability), np.log(1-probability)


# classifyNB:
#   words - vector of words of the tweet (binary vector)
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns (label of x according to the NB classification rule, confidence about the label)

# Note: you can also change the function definition if you wish to encapsulate all six log probs
# as one model; just make sure to follow through below

def classifyNB(words,logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive,
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
               logPriorPositive, logPriorNegative):

  probTweetPositive = np.exp(logPriorPositive + np.sum(logProbWordPresentGivenPositive[words==1]) + np.sum(logProbWordAbsentGivenPositive[words==0]))
  probTweetNegative = np.exp(logPriorNegative + np.sum(logProbWordPresentGivenNegative[words==1]) + np.sum(logProbWordAbsentGivenNegative[words==0]))

  if probTweetPositive > probTweetNegative:
    return 1, np.log(probTweetPositive / probTweetNegative)
  else:
    return -1, np.log(probTweetNegative / probTweetPositive)


# testNB: Classify all xTest
#   xTest - test data features
#   yTest - true label of test data
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns Average test error
def testNB(xTest, yTest,
           logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive,
           logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
           logPriorPositive, logPriorNegative):
  totalError = 0
  posConfidence = 0
  negConfidence = 0
  numPos = 0
  numNeg = 0

  for i, words in enumerate(xTest):
    classification, confidence = classifyNB(words,logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive,
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
               logPriorPositive, logPriorNegative)

    if classification != yTest[i]: totalError = totalError + 1
    if classification == 1:
      posConfidence += confidence
      numPos += 1
    else:
      negConfidence += confidence
      numNeg += 1

  # compute accuracy and confidence
  avgPosConfidence = posConfidence / numPos
  avgNegConfidence = negConfidence / numNeg
  accuracy = 1 - totalError/yTest.size
  return accuracy, avgPosConfidence, avgNegConfidence

# NtestNB: Run N iterations of testNB and gather average accuracy and confidence
#   n - number of iterations
#   X - occurance of words
#   y - lables of data
#   returns Average accuracy and confidence on positive and negative guesses for testNB over N iterations

def NTestNB(n, X, y):
  accuracy = 0
  posConfidence = 0
  negConfidence = 0
  for i in range(n):
    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)
    # gets the prior probabilities and conditional probabilities from our training data
    probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distrobutions(xTrain,yTrain)
    # converts probabilities to log form
    min_prob = 1/yTrain.shape[0]
    logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_log_probabilities(probWordGivenPositive, min_prob)
    logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_log_probabilities(probWordGivenNegative, min_prob)
    logPriorPositive, logPriorNegative = compute_log_probabilities(priorPositive, min_prob)

    # performs test and sums accuracy/confidence
    testAccuracy, testPosConfidence, testNegConfidence = testNB(xTest, yTest,
           logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive,
           logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
           logPriorPositive, logPriorNegative)

    accuracy += testAccuracy
    posConfidence += testPosConfidence
    negConfidence += testNegConfidence

  avgAccuracy = accuracy / n
  avgPosConfidence = posConfidence / n
  avgNegConfidence = negConfidence / n
  # print results of N tests
  print("--------------------NB--------------------")
  print(f"Over {n} tests our NB was {avgAccuracy * 100}% accuracy on average")
  print(f"Over {n} tests our NB scored {avgPosConfidence} confidence on positive guesses")
  print(f"Over {n} tests our NB scored {avgNegConfidence} confidence on negative guesses")

  return avgAccuracy, avgPosConfidence, avgNegConfidence

(3697, 5989) 29.0


---
---

**Part 2: An Alternate Model (50 pts)**

In Part 1, you calculated the probability of a tweet by incorporating both the probability of words present in the tweet $P\left(x^i_j=1 | +\right)$ and the probability of words absent from the tweet $P\left(x^i_j=0 | +\right)$.

Now, modify your code to *only* incorporate the probability of words present in the tweet $P\left(x^i_j=1 | +\right)$ (thus ignoring absent words).

Compare this to the original approach in Part 1. Follow reasonable experimental procedure and write up an explanation of the results you find.


In [ ]:
# classifyNBOnlyPresent:
#   words - vector of words of the tweet (binary vector)
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns (label of x according to the NB classification rule, confidence about the label)

# Note: you can also change the function definition if you wish to encapsulate all six log probs
# as one model; just make sure to follow through below

def classifyNBOnlyPresent(words,logProbWordPresentGivenPositive,
           logProbWordPresentGivenNegative, logPriorPositive, logPriorNegative):

  probTweetPositive = np.exp(logPriorPositive + np.sum(logProbWordPresentGivenPositive[words==1]))
  probTweetNegative = np.exp(logPriorNegative + np.sum(logProbWordPresentGivenNegative[words==1]))

  if probTweetPositive > probTweetNegative:
    return 1, np.log(probTweetPositive / probTweetNegative)
  else:
    return -1, np.log(probTweetNegative / probTweetPositive)


# testNBOnlyPresent: Classify all xTest
#   xTest - test data features
#   yTest - true label of test data
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns Average test error
def testNBOnlyPresent(xTest, yTest, logProbWordPresentGivenPositive,
           logProbWordPresentGivenNegative, logPriorPositive, logPriorNegative):
  totalError = 0
  posConfidence = 0
  negConfidence = 0
  numPos = 0
  numNeg = 0

  for i, words in enumerate(xTest):
    classification, confidence = classifyNBOnlyPresent(words,logProbWordPresentGivenPositive,
           logProbWordPresentGivenNegative, logPriorPositive, logPriorNegative)

    if classification != yTest[i]: totalError = totalError + 1
    if classification == 1:
      posConfidence += confidence
      numPos += 1
    else:
      negConfidence += confidence
      numNeg += 1

  # compute accuracy and confidence
  avgPosConfidence = posConfidence / numPos
  avgNegConfidence = negConfidence / numNeg
  accuracy = 1 - totalError/yTest.size
  return accuracy, avgPosConfidence, avgNegConfidence

# NtestNBOnlyPresent: Run N iterations of testNB and gather average accuracy and confidence
#   n - number of iterations
#   X - occurance of words
#   y - lables of data
#   returns Average accuracy and confidence on positive and negative guesses for testNBOnlyPresent over N iterations

def NTestNBOnlyPresent(n, X, y):
  accuracy = 0
  posConfidence = 0
  negConfidence = 0
  for i in range(n):
    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)
    # gets the prior probabilities and conditional probabilities from our training data
    probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distrobutions(xTrain,yTrain)
    # converts probabilities to log form
    min_prob = 1/yTrain.shape[0]
    logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_log_probabilities(probWordGivenPositive, min_prob)
    logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_log_probabilities(probWordGivenNegative, min_prob)
    logPriorPositive, logPriorNegative = compute_log_probabilities(priorPositive, min_prob)

    # performs test and sums accuracy/confidence
    testAccuracy, testPosConfidence, testNegConfidence = testNBOnlyPresent(xTest, yTest,
           logProbWordPresentGivenPositive,
           logProbWordPresentGivenNegative, logPriorPositive, logPriorNegative)

    accuracy += testAccuracy
    posConfidence += testPosConfidence
    negConfidence += testNegConfidence

  avgAccuracy = accuracy / n
  avgPosConfidence = posConfidence / n
  avgNegConfidence = negConfidence / n
  # print results of N tests
  print("--------------------NBOnlyPresent--------------------")
  print(f"Over {n} tests our NB was {avgAccuracy * 100}% accuracy on average")
  print(f"Over {n} tests our NB scored {avgPosConfidence} confidence on positive guesses")
  print(f"Over {n} tests our NB scored {avgNegConfidence} confidence on negative guesses")
  return avgAccuracy, avgPosConfidence, avgNegConfidence

## Part 2.B: Testing Only Absent

After seeing very little difference between Part 1 and Part 2 in our accuracy and confidence, I wanted to see if testing with only the absense of words would help shed some light on what was happening.

My hypothesis is that the presense of a word far outweighs the absense of a word, synonyms mean that 1 word could be very similar contextually to 10 or more words in our word bag. So if we only use absenses we should see far less accuracy and confidence.


In [ ]:
# classifyNBOnlyAbsent:
#   words - vector of words of the tweet (binary vector)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns (label of x according to the NB classification rule, confidence about the label)

# Note: you can also change the function definition if you wish to encapsulate all six log probs
# as one model; just make sure to follow through below

def classifyNBOnlyAbsent(words,logProbWordAbsentGivenPositive,
           logProbWordAbsentGivenNegative, logPriorPositive, logPriorNegative):

  probTweetPositive = np.exp(logPriorPositive + np.sum(logProbWordAbsentGivenPositive[words==0]))
  probTweetNegative = np.exp(logPriorNegative + np.sum(logProbWordAbsentGivenNegative[words==0]))

  if probTweetPositive > probTweetNegative:
    return 1, np.log(probTweetPositive / probTweetNegative)
  else:
    return -1, np.log(probTweetNegative / probTweetPositive)


# testNBOnlyAbsent: Classify all xTest
#   xTest - test data features
#   yTest - true label of test data
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns Average test error
def testNBOnlyAbsent(xTest, yTest, logProbWordAbsentGivenPositive,
           logProbWordAbsentGivenNegative, logPriorPositive, logPriorNegative):
  totalError = 0
  posConfidence = 0
  negConfidence = 0
  numPos = 0
  numNeg = 0

  for i, words in enumerate(xTest):
    classification, confidence = classifyNBOnlyAbsent(words,logProbWordAbsentGivenPositive,
           logProbWordAbsentGivenNegative, logPriorPositive, logPriorNegative)

    if classification != yTest[i]: totalError = totalError + 1
    if classification == 1:
      posConfidence += confidence
      numPos += 1
    else:
      negConfidence += confidence
      numNeg += 1

  # compute accuracy and confidence
  avgPosConfidence = posConfidence / numPos
  avgNegConfidence = negConfidence / numNeg
  accuracy = 1 - totalError/yTest.size

  return accuracy, avgPosConfidence, avgNegConfidence

# NtestNB: Run N iterations of testNB and gather average accuracy and confidence
#   n - number of iterations
#   X - occurance of words
#   y - lables of data
#   returns Average accuracy and confidence on positive and negative guesses for testNBOnlyAbsent over N iterations

def NTestNBOnlyAbsent(n, X, y):
  accuracy = 0
  posConfidence = 0
  negConfidence = 0
  for i in range(n):
    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)
    # gets the prior probabilities and conditional probabilities from our training data
    probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distrobutions(xTrain,yTrain)
    # converts probabilities to log form
    min_prob = 1/yTrain.shape[0]
    logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_log_probabilities(probWordGivenPositive, min_prob)
    logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_log_probabilities(probWordGivenNegative, min_prob)
    logPriorPositive, logPriorNegative = compute_log_probabilities(priorPositive, min_prob)

    # performs test and sums accuracy/confidence
    testAccuracy, testPosConfidence, testNegConfidence = testNBOnlyAbsent(xTest, yTest,
           logProbWordAbsentGivenPositive, logProbWordAbsentGivenNegative, logPriorPositive, logPriorNegative)

    accuracy += testAccuracy
    posConfidence += testPosConfidence
    negConfidence += testNegConfidence

  avgAccuracy = accuracy / n
  avgPosConfidence = posConfidence / n
  avgNegConfidence = negConfidence / n
  # print results of N tests
  print(f"--------------------NBOnlyAbsent--------------------")
  print(f"Over {n} tests our NB was {avgAccuracy * 100}% accuracy on average")
  print(f"Over {n} tests our NB scored {avgPosConfidence} confidence on positive guesses")
  print(f"Over {n} tests our NB scored {avgNegConfidence} confidence on negative guesses")
  return avgAccuracy, avgPosConfidence, avgNegConfidence

---
---
**Part 3: An Additional Experiment (50 pts)**

Implement an experiment to change the model, and report on the results of the experiment, comparing to your baseline model from Part 1 (and your alternate model from Part 2).

Choose one (and only one) of the following options.

*Please make clear which option you choose! (For example, by deleting the options you are not choosing.)*

**3.1: Option 1: Removing stop words**

Investigate the effect of removing the 25, 50, 100, and 200 most frequent words from the calculation.

**3.2: Option 2: Sample weights (3 bonus points for difficulty)**

Recall that the labels for each of our data points/samples came with a weight. This weight was based on the proportion of labelers that agreed on this label, so it serves as a kind of measure of confidence we should have in each data point. That is, a weight near 1 indicates everyone agreed on the same label. Whereas a weight below 0.5 means not even a majority agreed on the chosen label.

Devise a method for weighting samples, and use that method to recalculate the probability distributions.  Report the effect of weighting samples on the test set.

(Hint: Re-examine part 1.6 and think about how you would change this to make it pay more attention to data points with higher weights.)

**3.3: Option 3: Sticky terms (6 bonus points for difficulty)**

A "sticky term" is two words which are more likely to occur together than independently.

You can use "Pointwise Mutual Information" (PMI) to determine the stickiness, using: $PMI=\frac{P(w_1,w_2)}{P(w_1)P(w_2)}$.  For all pairs of <u>adjacent</u> words in the tweet corpus, find the top n pairs according to PMI and add them as additional features in your Naive Bayes Model.

Find the top 100, 200, 500 "sticky terms" and add these as features to the model.

(Note, you cannot use X to calculate the above joint distribution, since the above is about adjacent words. You will have to go back to the raw text.)

---

Remember, you need to *compare* your chosen option with your previous work. Just writing the code is not sufficient. Follow reasonable experimental procedure and write up a discussion of your results and why you think they turned out that way.

In [ ]:
# removeStopWords:
#   numWords - vector of words of the tweet (binary vector)
#   X - features to be split into train and test sets
#   returns (label of x according to the NB classification rule, confidence about the label)

# Note: you can also change the function definition if you wish to encapsulate all six log probs
# as one model; just make sure to follow through below

def removeStopWords(numWords, X):
  sumWords = np.sum(X, axis=0)
  sortedSumWords = np.sort(sumWords, kind="mergesort")
  cleanedX = X[:, sumWords < sortedSumWords[-numWords]]
  return cleanedX

# NtestNBRemoveStopWords: Run N iterations of testNB and gather average accuracy and confidence while removing stopwords
#   n - number of iterations
#   numWords - number of stop words to remove
#   X - occurance of words
#   y - lables of data
#   returns Average accuracy and confidence on positive and negative guesses for testNB over N iterations with {numWords} most common words removed
def NTestNBRemoveStopWords(n, numWords, X, y):
  X = removeStopWords(numWords, X)
  accuracy = 0
  posConfidence = 0
  negConfidence = 0
  for i in range(n):
    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)
    # gets the prior probabilities and conditional probabilities from our training data
    probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distrobutions(xTrain,yTrain)
    # converts probabilities to log form
    min_prob = 1/yTrain.shape[0]
    logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_log_probabilities(probWordGivenPositive, min_prob)
    logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_log_probabilities(probWordGivenNegative, min_prob)
    logPriorPositive, logPriorNegative = compute_log_probabilities(priorPositive, min_prob)

    # performs test and sums accuracy/confidence
    testAccuracy, testPosConfidence, testNegConfidence = testNB(xTest, yTest,
           logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive,
           logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
           logPriorPositive, logPriorNegative)

    accuracy += testAccuracy
    posConfidence += testPosConfidence
    negConfidence += testNegConfidence

  avgAccuracy = accuracy / n
  avgPosConfidence = posConfidence / n
  avgNegConfidence = negConfidence / n
  # print results of N tests
  print(f"--------------------NBRemove{numWords}Words--------------------")
  print(f"Over {n} tests our NB was {avgAccuracy * 100}% accuracy on average")
  print(f"Over {n} tests our NB scored {avgPosConfidence} confidence on positive guesses")
  print(f"Over {n} tests our NB scored {avgNegConfidence} confidence on negative guesses")

  return avgAccuracy, avgPosConfidence, avgNegConfidence

# Calls to all our various Naive Bayes Classifers

# Number of tests to run in each trial
numberOfTests = 10

# Baseline NB classifier
NBavgAccuracy, NBavgPosConfidence, NBavgNegConfidence = NTestNB(numberOfTests, X, y)

# NB classifier only considering present or absent words
NBavgAccuracyOnlyPresent, NBavgPosConfidenceOnlyPresent, NBavgNegConfidenceOnlyPresent = NTestNBOnlyPresent(numberOfTests, X, y)
NBavgAccuracyOnlyAbsent, NBavgPosConfidenceOnlyAbsent, NBavgNegConfidenceOnlyAbsent = NTestNBOnlyAbsent(numberOfTests, X, y)

#NB classifier with 25, 50, 100, 200 stopwords removed
NBavgAccuracyRemove25, NBavgPosConfidenceRemove25, NBavgNegConfidenceRemove25 = NTestNBRemoveStopWords(numberOfTests, 25, X, y)
NBavgAccuracyRemove25, NBavgPosConfidenceRemove25, NBavgNegConfidenceRemove25 = NTestNBRemoveStopWords(numberOfTests, 50, X, y)
NBavgAccuracyRemove25, NBavgPosConfidenceRemove25, NBavgNegConfidenceRemove25 = NTestNBRemoveStopWords(numberOfTests, 100, X, y)
NBavgAccuracyRemove25, NBavgPosConfidenceRemove25, NBavgNegConfidenceRemove25 = NTestNBRemoveStopWords(numberOfTests, 200, X, y)

--------------------NB--------------------
Over 100 tests our NB was 82.89999999999999% accuracy on average
Over 100 tests our NB scored 4.313647367811673 confidence on positive guesses
Over 100 tests our NB scored 3.5914769042035193 confidence on negative guesses
--------------------NBOnlyPresent--------------------
Over 100 tests our NB was 82.70270270270271% accuracy on average
Over 100 tests our NB scored 4.073734639229623 confidence on positive guesses
Over 100 tests our NB scored 3.5369117147371436 confidence on negative guesses
--------------------NBRemoveOnlyAbsent--------------------
Over 100 tests our NB was 60.01621621621622% accuracy on average
Over 100 tests our NB scored 0.45193207112944633 confidence on positive guesses
Over 100 tests our NB scored 0.12793843819888595 confidence on negative guesses
--------------------NBRemove25Words--------------------
Over 100 tests our NB was 81.99189189189188% accuracy on average
Over 100 tests our NB scored 3.7048214479143557 confid